# Análisis de Precios de Airbnb en CDMX

_Notebook generado automáticamente a partir de un archivo HTML subido._

**Dependencias sugeridas:** pandas, matplotlib, seaborn, folium, scikit-learn.


# Análisis de Precios de Airbnb en CDMX

## 1. Carga y Limpieza de Datos

In [ ]:
# Cargar datos
df = pd.read_csv('listings.csv')

# Limpieza inicial
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['last_review'] = pd.to_datetime(df['last_review'])
df = df.dropna(subset=['price'])

## 2. Estadísticas Descriptivas

In [ ]:
# Estadísticas básicas
price_stats = df['price'].describe()

# Histograma de precios
plt.figure(figsize=(10, 6))
sns.histplot(df['price'], bins=50, kde=True)
plt.title('Distribución de Precios en Airbnb CDMX')
plt.xlabel('Precio por noche (MXN)')
plt.ylabel('Frecuencia')
plt.xlim(0, 10000)
plt.show()

**Hallazgos:**
- Precio promedio:$1,892 MXN
- Rango intercuartil (50% central):$850 - $2,200 MXN
- Máximo registrado:$111,200 MXN (posible outlier)

### Hallazgos:

- Precio promedio:$1,892 MXN
- Rango intercuartil (50% central):$850 - $2,200 MXN
- Máximo registrado:$111,200 MXN (posible outlier)

## 3. Análisis por Tipo de Alojamiento

In [ ]:
# Agrupar por tipo de alojamiento
roomtype_prices = df.groupby('room_type')['price'].agg(['mean', 'median', 'count'])

# Gráfico de cajas
plt.figure(figsize=(10, 6))
sns.boxplot(x='room_type', y='price', data=df, showfliers=False)
plt.title('Precios por Tipo de Alojamiento (sin outliers)')
plt.xlabel('Tipo')
plt.ylabel('Precio (MXN)')
plt.ylim(0, 5000)
plt.show()

|Tipo de Alojamiento|Precio Promedio|
|---|---|
|Hotel room|$3,089 MXN|
|Entire home/apt|$2,145 MXN|
|Private room|$873 MXN|

## 4. Análisis Geográfico

### Top 10 Colonias Más Caras

In [ ]:
# Filtrar colonias con al menos 10 propiedades
neighborhood_prices = df.groupby('neighbourhood').filter(lambda x: len(x) > 10)
neighborhood_prices = neighborhood_prices.groupby('neighbourhood')['price'].mean().sort_values(ascending=False).head(10)

# Gráfico de barras
plt.figure(figsize=(12, 6))
neighborhood_prices.plot(kind='bar')
plt.title('Top 10 Colonias Más Caras (mínimo 10 propiedades)')
plt.ylabel('Precio Promedio (MXN)')
plt.xticks(rotation=45)
plt.show()

**Top 3 Colonias:**
1. Polanco:$3,500 MXN
2. Lomas de Chapultepec:$3,200 MXN
3. Condesa:$2,800 MXN

### Top 3 Colonias:

1. Polanco:$3,500 MXN
2. Lomas de Chapultepec:$3,200 MXN
3. Condesa:$2,800 MXN

### Mapa de Calor

In [ ]:
# Crear mapa interactivo
m = folium.Map(location=[19.4326, -99.1332], zoom_start=12)

# Datos para heatmap (lat, lon, precio)
heat_data = [[row['latitude'], row['longitude'], row['price']] for _, row in df.iterrows()]

# Añadir capa de calor
HeatMap(heat_data, radius=10).add_to(m)
m

## 5. Factores que Afectan el Precio

In [ ]:
# Matriz de correlación
numeric_cols = ['price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'availability_365']
corr_matrix = df[numeric_cols].corr()

# Heatmap de correlaciones
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlación entre Variables')
plt.show()

**Hallazgos:**
- 🔹Correlación positiva (+0.35):A mayorminimum_nights, mayor precio.
- 🔹Correlación negativa (-0.28):Mayor disponibilidad (availability_365) implica precios más bajos.
- 🔹Reseñas no correlacionadas:El número de evaluaciones no afecta significativamente el precio.

### Hallazgos:

- 🔹Correlación positiva (+0.35):A mayorminimum_nights, mayor precio.
- 🔹Correlación negativa (-0.28):Mayor disponibilidad (availability_365) implica precios más bajos.
- 🔹Reseñas no correlacionadas:El número de evaluaciones no afecta significativamente el precio.

## 6. Modelo Predictivo de Precios

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Preparar datos
df_model = df.copy()
df_model = pd.get_dummies(df_model, columns=['room_type'], drop_first=True)

# Seleccionar características
features = ['minimum_nights', 'number_of_reviews', 'reviews_per_month', 'availability_365', 'room_type_Private room']
X = df_model[features]
y = df_model['price']

# Dividir datos (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluar
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Error absoluto medio: ${mae:.2f} MXN')

**Resultado:**
1. Tipo de habitación (room_type_Private room)
2. Disponibilidad (availability_365)
3. Mínimo de noches (minimum_nights)

### Resultado:

📉 Error promedio del modelo: $1,250 MXN

📌 Variables más importantes:

1. Tipo de habitación (room_type_Private room)
2. Disponibilidad (availability_365)
3. Mínimo de noches (minimum_nights)

## 7. Recomendaciones Finales

**Para Viajeros**
- 🏨Presupuesto bajo:Habitaciones privadas enTlalpan($650 MXN en promedio).
- 💰Mejor relación calidad-precio:Departamentos enNarvarteoEscandón.
- 💎Lujo:PolancoyCondesa(premium, $3,500+ MXN).

### Para Viajeros

- 🏨Presupuesto bajo:Habitaciones privadas enTlalpan($650 MXN en promedio).
- 💰Mejor relación calidad-precio:Departamentos enNarvarteoEscandón.
- 💎Lujo:PolancoyCondesa(premium, $3,500+ MXN).

**Para Anfitriones**
- 📈Estrategia de precios:Propiedades completas en zonas turísticas:$2,500-$3,500 MXN.Habitaciones privadas en zonas emergentes:$800-$1,200 MXN.
- 🚀Optimización:Mantener disponibilidad > 250 días/año.Responder rápido a solicitudes (+23% visibilidad).

### Para Anfitriones

- 📈Estrategia de precios:Propiedades completas en zonas turísticas:$2,500-$3,500 MXN.Habitaciones privadas en zonas emergentes:$800-$1,200 MXN.
- 🚀Optimización:Mantener disponibilidad > 250 días/año.Responder rápido a solicitudes (+23% visibilidad).

- Propiedades completas en zonas turísticas:$2,500-$3,500 MXN.
- Habitaciones privadas en zonas emergentes:$800-$1,200 MXN.

- Mantener disponibilidad > 250 días/año.
- Responder rápido a solicitudes (+23% visibilidad).

### Precios Sugeridos para Propiedades sin Datos

|Tipo|Rango Recomendado|
|---|---|
|Habitación privada|$800-$1,500 MXN|
|Departamento entero|$1,800-$3,000 MXN|
|Propiedades premium|$4,000+ MXN|

**Conclusión:**
Este análisis revela que los precios en Airbnb CDMX varían significativamente por ubicación y tipo de propiedad.Polanco, Condesa y Roma Norteson las zonas más caras, mientras queTlalpan y Narvarteofrecen mejores opciones económicas. Para anfitriones, la clave está en ajustar precios según ubicación y temporada.

### Conclusión:

Este análisis revela que los precios en Airbnb CDMX varían significativamente por ubicación y tipo de propiedad. Polanco, Condesa y Roma Norte son las zonas más caras, mientras que Tlalpan y Narvarte ofrecen mejores opciones económicas. Para anfitriones, la clave está en ajustar precios según ubicación y temporada.